In [1]:
# import necessary modules
from datetime import datetime
import pandas as pd
import psycopg2

In [2]:
# print start of the job
start_time = datetime.now()
print('Data Transformation job started at at {}'.format(start_time))

Data Transformation job started at at 2023-05-24 21:02:29.421487


In [3]:
# Establish a connection to the PostgreSQL server
host = "localhost"
user = "postgres"
password = "mm4easy1T"
conn = psycopg2.connect(host=host,  user=user, password=password)

In [4]:
df_raw = pd.read_sql(f"SELECT * FROM pcard_expenditures.raw_data_transactions",conn)
print(df_raw.shape)
df_raw.head()

(394217, 17)


,Division,BatchTransactionID,TransactionDate,CardPostingDate,MerchantName,TransactionAmount,TransactionCurrency,OriginalAmount,OriginalCurrency,GLAccount,GLAccountDescription,CostCenterWBSElementOrder,CostCenterWBSElementOrderDescription,MerchantType,MerchantTypeDescription,Purpose,SourceFileName
0,PUBLIC HEALTH,4608-1,2017-06-15,2017-06-16,PAYPAL *OBC2012,50.0,CAD,50.0,CAD,4760,MEMBERSHIP FEES,PH3071,MATERNAL INFANT HEALTH PROGRAM SUPPORT,8641.0,"Associations - Civic, Social, and Frater",MEMBERSHIP FEE,PCard Expenses_201706.xlsx
1,PUBLIC HEALTH,4617-1,2017-06-21,2017-06-23,CHNC,423.75,CAD,423.75,CAD,4256,CONFERENCES/SEMINARS - REGISTRATION FEES,PH3071,MATERNAL INFANT HEALTH PROGRAM SUPPORT,8641.0,"Associations - Civic, Social, and Frater",CHNC CONFERENCE,PCard Expenses_201706.xlsx
2,PUBLIC HEALTH,4621-1,2017-06-26,2017-06-27,POST MD-CPD-UOFT,2601.0,CAD,2601.0,CAD,4256,CONFERENCES/SEMINARS - REGISTRATION FEES,PH3071,MATERNAL INFANT HEALTH PROGRAM SUPPORT,8220.0,"Colleges, Universities, Professional Sch",UOFT CONFERENCE,PCard Expenses_201706.xlsx
3,PUBLIC HEALTH,4626-1,2017-06-28,2017-06-29,EVENTBRITE/PERINATALMO,480.66,CAD,480.66,CAD,4256,CONFERENCES/SEMINARS - REGISTRATION FEES,PH3071,MATERNAL INFANT HEALTH PROGRAM SUPPORT,7922.0,Theatrical Producers (except Motion Pict,REGISTRATION FEES,PCard Expenses_201706.xlsx
4,ECONOMIC DEVELOPMENT & CULTURE,4590-1,2017-06-02,2017-06-05,LEE VALLEY - DOWNTOWN,38.31,CAD,38.31,CAD,2600,RECREATIONAL & EDUCATIONAL SUPPLIES,AH0073,HS-MH-EDUCATION/OUTRCH-FORT YORK,5072.0,Hardware Equipment and Supplies,LEMON RASPS,PCard Expenses_201706.xlsx


### Infer Tables

### TRANSACTIONS

In [5]:
df_raw.columns

Index(['Division', 'BatchTransactionID', 'TransactionDate', 'CardPostingDate',
       'MerchantName', 'TransactionAmount', 'TransactionCurrency',
       'OriginalAmount', 'OriginalCurrency', 'GLAccount',
       'GLAccountDescription', 'CostCenterWBSElementOrder',
       'CostCenterWBSElementOrderDescription', 'MerchantType',
       'MerchantTypeDescription', 'Purpose', 'SourceFileName'],
      dtype='object')

In [6]:
# transactions
transactions = df_raw.groupby(['BatchTransactionID','TransactionDate', 'CardPostingDate',
                               'TransactionAmount', 'TransactionCurrency', 'OriginalAmount','OriginalCurrency',
                               'GLAccount','CostCenterWBSElementOrder','MerchantName',
                               'Division','Purpose'
                              ]).size().reset_index(name='Count').drop(columns='Count')
print(transactions.shape)
transactions.head()

(394217, 12)


,BatchTransactionID,TransactionDate,CardPostingDate,TransactionAmount,TransactionCurrency,OriginalAmount,OriginalCurrency,GLAccount,CostCenterWBSElementOrder,MerchantName,Division,Purpose
0,1263-1,2010-12-31,2011-01-04,1508.55,CAD,1508.55,CAD,4520,AH0007,PAYPAL *KENNETHROBE,ECONOMIC DEVELOPMENT & CULTURE,Audio Visual Equipment Rental Set-up & Strike ...
1,1263-11,2010-12-30,2011-01-04,57.17,CAD,57.17,CAD,2600,D3A083,PARTY PACKAGERS (WILLI,LONG TERM CARE HOMES,Program Supplies
2,1263-12,2010-12-30,2011-01-04,111.75,CAD,111.75,CAD,2860,D3L083,CDN TIRE STORE #00070,LONG TERM CARE HOMES,Large Christmas tote
3,1263-14,2010-12-30,2011-01-04,50.01,CAD,50.01,CAD,4820,ED0057,JUMP CAFE BAR,ECONOMIC DEVELOPMENT & CULTURE,BUS MTG EXP - ASAIN DEVELOPMENT BANK
4,1263-15,2010-12-31,2011-01-04,22.59,CAD,22.59,CAD,2010,GV0028,RCSS 1077 QPS,CITY CLERK'S OFFICE,Superstore


In [7]:
transactions.columns

Index(['BatchTransactionID', 'TransactionDate', 'CardPostingDate',
       'TransactionAmount', 'TransactionCurrency', 'OriginalAmount',
       'OriginalCurrency', 'GLAccount', 'CostCenterWBSElementOrder',
       'MerchantName', 'Division', 'Purpose'],
      dtype='object')

In [8]:
# we now want to save our data in the database
# we get all the records into individual rows and save them at once
trxn_rows = []
for index, row in transactions.iterrows():
    record = (row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],
              row[9],row[10],row[11])
    trxn_rows.append(record)
trxn_rows[:2]

[('1263-1',
  '2010-12-31',
  '2011-01-04',
  '1508.55',
  'CAD',
  '1508.55',
  'CAD',
  '4520',
  'AH0007',
  'PAYPAL *KENNETHROBE',
  'ECONOMIC DEVELOPMENT & CULTURE',
  'Audio Visual Equipment Rental Set-up & Strike Serv'),
 ('1263-11',
  '2010-12-30',
  '2011-01-04',
  '57.17',
  'CAD',
  '57.17',
  'CAD',
  '2600',
  'D3A083',
  'PARTY PACKAGERS (WILLI',
  'LONG TERM CARE HOMES',
  'Program Supplies')]

In [9]:
# create a table
try:
    # disable autocommit
    conn.autocommit = False
    # Create a cursor object
    cursor = conn.cursor()
    
    table_name = "pcard_expenditures.pcard_transactions"    
    
    # transaction 1: create the table
    sql = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        "BatchTransactionID" VARCHAR(250), 
        "TransactionDate" DATE, 
        "CardPostingDate" DATE, 
        "TransactionAmount" VARCHAR(250), 
        "TransactionCurrency" VARCHAR(250), 
        "OriginalAmount" VARCHAR(250), 
        "OriginalCurrency" VARCHAR(250), 
        "GLAccount" VARCHAR(250), 
        "CostCenterWBSElementOrder" VARCHAR(250), 
        "MerchantName" VARCHAR(250), 
        "Division" VARCHAR(250), 
        "Purpose" VARCHAR(250)
    )
    """
    cursor.execute(sql) 
    
    print(f'{table_name} successfully created')
    
    # transaction 2: truncate the table
    sql = f'TRUNCATE TABLE {table_name}'
    cursor.execute(sql)
    
    # transaction 3: save the new data
    sql = f"""
        INSERT INTO {table_name} ("BatchTransactionID",  "TransactionDate", "CardPostingDate",
         "TransactionAmount", "TransactionCurrency","OriginalAmount", "OriginalCurrency","GLAccount",
         "CostCenterWBSElementOrder", "MerchantName","Division", "Purpose")
     VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """
    
    
    cursor.executemany(sql,trxn_rows)
    # Commit the changes
    conn.commit()
    
    # Close the cursor and the connection
    cursor.close()

    print(f'{len(transactions)} rows successfully saved in {table_name}')
    
except Exception as e:
    print(f'An error occurred: {e}')

An error occurred: set_session cannot be used inside a transaction


### MERCHANTS

In [10]:
# merchants
merchants = df_raw.groupby(['MerchantName','MerchantType']).size().reset_index(name='Count').drop(columns='Count')
print(merchants.shape)
merchants.head()

(29780, 2)


,MerchantName,MerchantType
0,ONTARS ASSOC OF ONTAR,8641.0
1,"""77"" AUTO PARTS",5533.0
2,#0345 1052 CLONSILLA,5541.0
3,#061 MARK S,5651.0
4,#068 MARK S,5651.0


In [11]:
# remove the digits after decimal point
merchants['MerchantType'] = merchants['MerchantType'].apply(lambda x: x.replace('.0',''))
merchants.head()

,MerchantName,MerchantType
0,ONTARS ASSOC OF ONTAR,8641
1,"""77"" AUTO PARTS",5533
2,#0345 1052 CLONSILLA,5541
3,#061 MARK S,5651
4,#068 MARK S,5651


In [12]:
# we now want to save our data in the database
# we get all the records into individual rows and save them at once
merc_rows = []
for index, row in merchants.iterrows():
    record = (row[0],row[1])
    merc_rows.append(record)
merc_rows[:2]

[(' ONTARS ASSOC OF ONTAR', '8641'), ('"77" AUTO PARTS', '5533')]

In [13]:
# create a table
try:
    # disable autocommit
    conn.autocommit = False
    # Create a cursor object
    cursor = conn.cursor()
    
    table_name = "pcard_expenditures.pcard_merchants"    
    
    # transaction 1: create the table
    sql = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        "MerchantName" VARCHAR(250), 
        "MerchantType" VARCHAR(10), 
    )
    """
    cursor.execute(sql) 
    # Close the cursor and the connection
    cursor.close()
    print(f'{table_name} successfully created')
    
    # transaction 2: truncate the table
    sql = f'TRUNCATE TABLE {table_name}'
    cursor.execute(sql)
    
    # transaction 3: save the new data
    sql = f"""
        INSERT INTO {table_name} ("MerchantName",  "MerchantType") VALUES (%s,%s)
    """
    
    
    cursor.executemany(sql,merc_rows)
    # Commit the changes
    conn.commit()
    
    # Close the cursor and the connection
    cursor.close()

    print(f'{len(merchants)} rows successfully saved in {table_name}')
    
except Exception as e:
    print(f'An error occurred: {e}')

An error occurred: set_session cannot be used inside a transaction


### MERCHANT TYPES

In [14]:
# merchants
merchant_types = df_raw.groupby(['MerchantType','MerchantTypeDescription']).size().reset_index(name='Count').drop(columns='Count')
print(merchant_types.shape)
merchant_types.head()

(364, 2)


,MerchantType,MerchantTypeDescription
0,0.0,Default
1,1520.0,General Contractors - Residential and Co
2,1711.0,"Air Conditioning, Heating, and Plumbing"
3,1731.0,Electrical Contractors
4,1740.0,"Insulation, Masonry, Plastering, Stonewo"


In [15]:
# remove the digits after decimal point
merchant_types['MerchantType'] = merchant_types['MerchantType'].apply(lambda x: x.replace('.0',''))
merchant_types.head()

,MerchantType,MerchantTypeDescription
0,0,Default
1,1520,General Contractors - Residential and Co
2,1711,"Air Conditioning, Heating, and Plumbing"
3,1731,Electrical Contractors
4,1740,"Insulation, Masonry, Plastering, Stonewo"


In [16]:
# we now want to save our data in the database
# we get all the records into individual rows and save them at once
merc_type_rows = []
for index, row in merchant_types.iterrows():
    record = (row[0],row[1])
    merc_type_rows.append(record)
merc_type_rows[:2]

[('0', 'Default'), ('1520', 'General Contractors - Residential and Co')]

In [17]:
# create a table
try:
    # disable autocommit
    conn.autocommit = False
    # Create a cursor object
    cursor = conn.cursor()
    
    table_name = "pcard_expenditures.pcard_merchant_types"    
    
    # transaction 1: create the table
    sql = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        "MerchantType" VARCHAR(10), 
        "MerchantTypeDescription" VARCHAR(250), 
    )
    """
    cursor.execute(sql) 
    # Close the cursor and the connection
    cursor.close()
    print(f'{table_name} successfully created')
    
    # transaction 2: truncate the table
    sql = f'TRUNCATE TABLE {table_name}'
    cursor.execute(sql)
    
    # transaction 3: save the new data
    sql = f"""
        INSERT INTO {table_name} ("MerchantType",  "MerchantTypeDescription") VALUES (%s,%s)
    """
    
    
    cursor.executemany(sql,merc_type_rows)
    # Commit the changes
    conn.commit()
    
    # Close the cursor and the connection
    cursor.close()

    print(f'{len(merchant_types)} rows successfully saved in {table_name}')
    
except Exception as e:
    print(f'An error occurred: {e}')

An error occurred: set_session cannot be used inside a transaction


### GL ACCOUNTS

In [18]:
df_raw.columns

Index(['Division', 'BatchTransactionID', 'TransactionDate', 'CardPostingDate',
       'MerchantName', 'TransactionAmount', 'TransactionCurrency',
       'OriginalAmount', 'OriginalCurrency', 'GLAccount',
       'GLAccountDescription', 'CostCenterWBSElementOrder',
       'CostCenterWBSElementOrderDescription', 'MerchantType',
       'MerchantTypeDescription', 'Purpose', 'SourceFileName'],
      dtype='object')

In [19]:
# merchants
gl_accounts = df_raw.groupby(['GLAccount','GLAccountDescription']).size().reset_index(name='Count').drop(columns='Count')
print(gl_accounts.shape)
gl_accounts.head()

(931, 2)


,GLAccount,GLAccountDescription
0,****,MULTIPLE G/L ACCOUNTS USED
1,*****,ADVERTISING & PROMOTION
2,*****,AGRICULTURE & BOTANICAL SUPPLIES
3,*****,ANIMAL CARE SUPPLIES
4,*****,ASPHALT


In [20]:
# we now want to save our data in the database
# we get all the records into individual rows and save them at once
gl_rows = []
for index, row in gl_accounts.iterrows():
    record = (row[0],row[1])
    gl_rows.append(record)
gl_rows[:2]

[('****', 'MULTIPLE G/L ACCOUNTS USED'), ('*****', 'ADVERTISING & PROMOTION')]

In [21]:
# create a table
try:
    # disable autocommit
    conn.autocommit = False
    # Create a cursor object
    cursor = conn.cursor()
    
    table_name = "pcard_expenditures.pcard_gl_accounts"    
    
    # transaction 1: create the table
    sql = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        "GLAccount" VARCHAR(250), 
        "GLAccountDescription" VARCHAR(250), 
    )
    """
    cursor.execute(sql) 
    # Close the cursor and the connection
    cursor.close()
    print(f'{table_name} successfully created')
    
    # transaction 2: truncate the table
    sql = f'TRUNCATE TABLE {table_name}'
    cursor.execute(sql)
    
    # transaction 3: save the new data
    sql = f"""
        INSERT INTO {table_name} ("GLAccount",  "GLAccountDescription") VALUES (%s,%s)
    """
    
    
    cursor.executemany(sql,gl_rows)
    # Commit the changes
    conn.commit()
    
    # Close the cursor and the connection
    cursor.close()

    print(f'{len(gl_accounts)} rows successfully saved in {table_name}')
    
except Exception as e:
    print(f'An error occurred: {e}')

An error occurred: set_session cannot be used inside a transaction


### WBS Elements

In [22]:
# merchants
wbs = df_raw.groupby(['CostCenterWBSElementOrder','CostCenterWBSElementOrderDescription']).size().reset_index(name='Count').drop(columns='Count')
print(wbs.shape)
wbs.head()

(15828, 2)


,CostCenterWBSElementOrder,CostCenterWBSElementOrderDescription
0,*****,BLDG REPAIRS & RENOVATIONS - VARIOUS LOCATIONS
1,*****,BLK COUNC SAL&BEN-C OOTES
2,*****,COURT FEES
3,*****,MULTIPLE COST CENTRES ASSIGNED
4,*****,MULTIPLE COST CENTRES OR WBS ELEMENTS


In [23]:
# we now want to save our data in the database
# we get all the records into individual rows and save them at once
wbs_rows = []
for index, row in wbs.iterrows():
    record = (row[0],row[1])
    wbs_rows.append(record)
wbs_rows[:2]

[('*****', 'BLDG REPAIRS & RENOVATIONS - VARIOUS LOCATIONS'),
 ('*****', 'BLK COUNC SAL&BEN-C OOTES')]

In [24]:
# create a table
try:
    # disable autocommit
    conn.autocommit = False
    # Create a cursor object
    cursor = conn.cursor()
    
    table_name = "pcard_expenditures.pcard_wbs_elements"    
    
    # transaction 1: create the table
    sql = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        "CostCenterWBSElementOrder" VARCHAR(250), 
        "CostCenterWBSElementOrderDescription" VARCHAR(250), 
    )
    """
    cursor.execute(sql) 
    # Close the cursor and the connection
    cursor.close()
    print(f'{table_name} successfully created')
    
    # transaction 2: truncate the table
    sql = f'TRUNCATE TABLE {table_name}'
    cursor.execute(sql)
    
    # transaction 3: save the new data
    sql = f"""
        INSERT INTO {table_name} ("CostCenterWBSElementOrder",  "CostCenterWBSElementOrderDescription") VALUES (%s,%s)
    """
    
    
    cursor.executemany(sql,wbs_rows)
    # Commit the changes
    conn.commit()
    
    # Close the cursor and the connection
    cursor.close()

    print(f'{len(wbs)} rows successfully saved in {table_name}')
    
except Exception as e:
    print(f'An error occurred: {e}')

An error occurred: set_session cannot be used inside a transaction


In [25]:
end_time = datetime.now()
print(f"""
Data Transformation job started at {start_time}.
Data Transformation job finished execution at {end_time}.
Total time taken is for Data Transformation is {end_time-start_time}
""")


Data Transformation job started at 2023-05-24 21:02:29.421487.
Data Transformation job finished execution at 2023-05-24 21:03:19.756883.
Total time taken is for Data Transformation is 0:00:50.335396

